# Introducción
## Objetivo
Este programa se tuiliza para asignar grupos a estudiantes siguiendo las prioridades:
* Todos los estudiantes (o la mayor cantidad posible), deberían ser asignados a un grupo que hayan marcado como deseable
* Los estudiantes que presentan certificado laboral/deportivo recibem prioridad al momento de la asignación grupos

## Resultado
El programa genera como archivo de salida una tabla en la que se indica el grupo asignado a cada estudiante. En una columna adicional se indica si el turno asignado se encuentra entre los elegidos por el estudiante.

## Reglas
### Sobre los grupos
* Existe un numero $n$ de grupos
* Cada grupo $n_i$ tiene un numero determinado de cupos $c_i$
* Los grupos estan agregados en $S$ *supergrupos* que pueden corresponder a turnos (Matutino, Vespertino, Nocturno), salones de clase, centros de estudio, etc. Cada grupo $n_i$ pertenece a un único supergrupo $S_j$

### Sobre los estudiantes
* Existen $m$ estudiantes. Identificados por un número de cedula o pasaporte.
* Cada estudiante debe elegir al menos 1 grupo en al menos 2 supergrupos diferentes (Todos los estudiantes presentan como mínimo 2 opciones de grupo y 2 opciones de supergrupo).
* Los estudiantes pueden presentar certificado laboral o deportivo. En este caso los estudiantes tienen prioridad en la selección de grupos

# Parametros
A continuación se detallan los parametros necesarios para ejecutar el programa. Algunos de estos pueden tener un valor por defecto.

In [143]:
# Parametros del programa
# Parametros de los grupos
archivo_grupos = 'Grupos_Disponibles.txt'
columna_supergrupos = 'Turno'
columna_min_estudiantes = 'Min estudiantes'
columna_max_estudiantes = 'Max estudiantes'

# Parametros de los estudiantes
archivo_estudiantes = 'Formulario_Estudiantes.txt'
columnas_supergrupos = 'Grupos mañana___Grupos tarde___Grupos noche'
columnas_certificados = 'certificado'
columnas_identificacion = 'Cedula___pasaporte'

In [57]:
# Modulos
import numpy as np
import pandas as pd
import re

In [5]:
grupos = pd.read_csv(archivo_grupos, sep='\t', index_col=0)
grupos.head()

,Turno,Min Estudiantes,Max Estudiantes
ID grupo,,,
1,Turno 1,1,35
2,Turno 1,1,35
3,Turno 1,1,35
4,Turno 1,1,35
5,Turno 1,1,35


In [119]:
estudiantes = pd.read_csv(archivo_estudiantes, sep='\t')
estudiantes.head()

,Marca temporal,Correo,Cedula,pasaporte,Nacionali,NOMBRE completo,APELLIDOS,Fecha nacimiento,depto 2023,localidad 2024,barrio,udelar previa,otros estud,Grupos mañana,Grupos tarde,Grupos noche,TRABAJA,Deporte,certificado
0,26/02/2024 12:48:29,florencialado01@gmail.com,5165495,NaN,Uruguaya,Florencia,Lado Casaglia,03/04/2002,CANELONES,Canelones La Floresta,No vivo en montevideo,NO,NO,NaN,27,NaN,No,NO,NaN
1,26/02/2024 14:04:46,florencialado01@gmail.com,5165495,NaN,Uruguaya,Florencia,Lado Casaglia,03/04/2002,CANELONES,La floresta canelones,No vivo en montevideo,SI,NO,NaN,14,NaN,No,NO,NaN
2,26/02/2024 15:59:44,diegoodera9@gmail.com,44281309,NaN,Uruguaya,Diego Maximiliano,Odera Piñeyro,14/02/1992,MONTEVIDEO,Montevideo,Guaycuru 2884(barrio Reducto),SI,NO,NaN,9,15,SI,NO,https://drive.google.com/open?id=1A4_v2vYuuRva...
3,27/02/2024 21:53:05,diegoodera9@gmail.com,44281309,NaN,Uruguayo,Diego Maximiliano,Odera Piñeyro,14/02/1992,MONTEVIDEO,Montevideo,Reducto,SI,NO,NaN,66,15,SI,NO,https://drive.google.com/open?id=1VlUhv9N7JFFB...
4,26/02/2024 15:18:35,claumansilla46197@gmail.com,46197390,NaN,Oriental,Claudia Mariana,Mansilla Goicoechea,24/08/1993,MALDONADO,Maldonado,No,SI,NO,1,9,15,SI,NO,NaN


In [144]:
# parsear argumentos
super_group = columna_supergrupos
max_stds_col = columna_max_estudiantes
min_stds_col = columna_min_estudiantes

super_group_cols = columnas_supergrupos.split('___')
cert_cols = columnas_certificados.split('___')
id_cols = columnas_identificacion.split('___')

In [120]:
# identificar estudiantes
id_cols = (~estudiantes[id_cols].isna()).sum(axis=0).sort_values(ascending=False).index
identification = estudiantes[[id_cols[0]]].copy().rename(columns={id_cols[0]:'Id'})
identification['Doc_type'] = id_cols[0]
for col in id_cols[1:]:
    missing_ids = identification.Id.isna()
    identification.loc[missing_ids, 'Id'] = estudiantes.loc[missing_ids, col].values
    identification.loc[missing_ids, 'Doc_type'] = col
estudiantes = pd.concat((estudiantes, identification), axis=1)

# filtrar registros sin grupos asignados
sin_grupos = (~estudiantes[super_group_cols].isna()).sum(axis=1) == 0
registros_filtrados = estudiantes.loc[sin_grupos]
estudiantes = estudiantes.loc[~sin_grupos]

In [132]:
# construir tabla de cupos
places_tab = pd.DataFrame(False, index=estudiantes.Id.unique(), columns=grupos.index)
places_tab.head()

for ID, student_tab in estudiantes.groupby('Id'):
    student_groups = ','.join(student_tab[super_group_cols].fillna('').agg(','.join, axis=1).values)
    student_groups = re.sub('^,', '', re.sub(',$', '', re.sub(',,+', ',', re.sub(' ','', student_groups))))
    student_groups = np.unique(student_groups.split(',')).astype(int)
    places_tab.loc[ID, student_groups] = True

weird_groups = places_tab.drop(columns=grupos.index)
places_tab = places_tab[grupos.index]

In [152]:
# determinar estudiantes prioritarios
certified = pd.Series(False, index = places_tab.index)
certified[estudiantes.loc[~estudiantes[cert_cols].isna().values, 'Id'].unique()] = True

# Como funciona el programa
1) Se suma el numero de estudiantes que

## ideaza
Rastrear progreso con una grafica del excedente de estudiantes en cada grupo para cada iteración
* Eje x : cursos, ordenados en orden ascendente por cantidad de estudiantes sobrantes en t-0
* Eje y : numero de estudiantes sobrantes
* Multiples lineas, cada una representa una iteración, debería indicar como van bajando los estudiantes sobrantes en el resto de los grupos